In [1]:
%load_ext autoreload
%autoreload 2

import os
import sys
from glob import glob

import numpy as np
import pandas as pd
idx = pd.IndexSlice
import matplotlib.pyplot as plt
from matplotlib.ticker import MaxNLocator

import copy

import geopandas as gpd

from sklearn.metrics import mean_absolute_error

from sklearn.linear_model import LinearRegression
from sklearn.neural_network import MLPRegressor

import gpflow
import tensorflow as tf
import sys
code_dir = '/cluster/home/kheuto01/code/zero-inflated-gp/'
sys.path.append(code_dir)
from math import radians, cos, sin, asin, sqrt
from onoffgpf import OnOffSVGP, OnOffLikelihood

import pickle

from math import radians, cos, sin, asin, sqrt

import copy
import sys

import geopandas as gpd
import numpy as np

import gpflow


code_dir = '/cluster/home/kheuto01/code/zero-inflated-gp/'
sys.path.append(code_dir)

code_dir = '/cluster/home/kheuto01/code/opioid-overdose-models/'
sys.path.append(code_dir)

from onoffgpf import OnOffSVGP, OnOffSVGPPoiMC, OnOffLikelihood
gpflow.config.default_float()


from zinf_gp.metrics import normcdf, fixed_top_X

import numpy as np 
import matplotlib.pyplot as plt 
import pandas as pd 
import contextily as ctx 
import geopandas as gpd 
import os 
from mpl_toolkits.axes_grid1 import make_axes_locatable


import matplotlib
matplotlib.style.use('default')
font = {
        'size'   : 22}

matplotlib.rc('font', **font)

2023-02-28 19:07:33.841638: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F AVX512_VNNI FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-02-28 19:07:33.968783: I tensorflow/core/util/util.cc:169] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2023-02-28 19:07:33.974158: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2023-02-28 19:07:33.974172: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if yo

In [2]:

data_dir='/cluster/tufts/hugheslab/datasets/NSF_OD/results_20220606_update/'

log_dir='/cluster/tufts/hugheslab/kheuto01/opioid/logs/fix_two/'

run_template = '{time}_{loc}_{model}_{start_year}_{cov}_{num_inducing}_{lr}'

In [3]:
time = 'annual'
loc = 'tract'
model = 'normal'
start_year = 2000
cov = '-auto'
num_inducing = '400'
lr = 0.05

In [18]:
test_years = 2
geography_col='geoid'
timestep_col='timestep'
outcome_col='deaths'

# test y always comes from quarterly tract
y_timesteps_per_year = 4
file_name = f'clean_quarter_tract'
data_path = os.path.join(data_dir, file_name)

x_idx_cols = [geography_col, 'lat','lon', timestep_col,
      'theme_1_pc', 'theme_2_pc', 'theme_3_pc', 'theme_4_pc',
      'svi_pctile',
      'neighbor_t', 'self_t-1']
y_idx_cols = [geography_col, timestep_col, outcome_col]
features_only = ['lat','lon', timestep_col,
                 'theme_1_pc', 'theme_2_pc', 'theme_3_pc', 'theme_4_pc',
                 'svi_pctile',
                 'neighbor_t', 'self_t-1']

data_gdf = gpd.read_file(data_path)

last_train_year = 2018
train_x_qtr = data_gdf[(data_gdf['year'] <= last_train_year)][x_idx_cols]

train_y_qtr = data_gdf[(data_gdf['year'] <= last_train_year)][y_idx_cols]

all_y_qtr = data_gdf[y_idx_cols]
test_x_qtr = data_gdf[(data_gdf['year'] > last_train_year) &
                  (data_gdf['year'] <= last_train_year+test_years)][x_idx_cols]

test_y = data_gdf[(data_gdf['year'] > last_train_year) &
                  (data_gdf['year'] <= last_train_year+test_years)][y_idx_cols]
starting_y_timestep = int(test_y[timestep_col].min())

sorted_y_timesteps = test_y[timestep_col].unique()
sorted_y_timesteps.sort()

x_timesteps_per_year = {'quarter':4, 'semi':2,'annual':1}[time]

file_name = f'clean_{time}_{loc}'
data_path = os.path.join(data_dir, file_name)

data_gdf = gpd.read_file(data_path)

train_x = data_gdf[(data_gdf['year'] <= last_train_year) &
                   (data_gdf['year'] >= start_year)][x_idx_cols]
train_y = data_gdf[(data_gdf['year'] <= last_train_year) &
                   (data_gdf['year'] >= start_year)][y_idx_cols]

test_x = data_gdf[(data_gdf['year'] > last_train_year) &
                  (data_gdf['year'] <= last_train_year+test_years)][x_idx_cols]
test_y_yr = data_gdf[(data_gdf['year'] > last_train_year) &
                  (data_gdf['year'] <= last_train_year+test_years)][y_idx_cols]
starting_x_timestep = int(test_x[timestep_col].min())

test_timesteps_per_year = max(y_timesteps_per_year, x_timesteps_per_year)
test_timesteps = test_timesteps_per_year*test_years

x_repeats = int(test_timesteps_per_year/x_timesteps_per_year)
y_repeats = int(test_timesteps_per_year/y_timesteps_per_year)

sorted_x_timesteps = test_x[timestep_col].unique()
sorted_x_timesteps.sort()

x_timesteps = [timestep  for timestep in sorted_x_timesteps for _ in range(x_repeats)]

y_timesteps = [timestep  for timestep in sorted_y_timesteps for _ in range(y_repeats)]



In [5]:
this_run = run_template.format(time=time,loc=loc,model=model,start_year=start_year,
                                                           cov=cov,
                                                           num_inducing=num_inducing,lr=lr)
with open(os.path.join(log_dir,this_run,'model.mod'),'rb') as f:
        predictor = pickle.load(f)
with open(os.path.join(log_dir,this_run,'stats.csv'),'rb') as f:
    stats = pd.read_csv(f)

2023-03-01 00:24:52.181485: E tensorflow/stream_executor/cuda/cuda_driver.cc:265] failed call to cuInit: CUDA_ERROR_NO_DEVICE: no CUDA-capable device is detected
2023-03-01 00:24:52.181530: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:169] retrieving CUDA diagnostic information for host: p1cmp078.pax.tufts.edu
2023-03-01 00:24:52.181540: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:176] hostname: p1cmp078.pax.tufts.edu
2023-03-01 00:24:52.181688: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:200] libcuda reported version is: 510.47.3
2023-03-01 00:24:52.181720: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:204] kernel reported version is: 510.47.3
2023-03-01 00:24:52.181726: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:310] kernel version seems to match DSO: 510.47.3
2023-03-01 00:24:52.182179: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the fo

In [6]:
new_tract_shapefile = os.path.join(os.path.dirname(os.path.dirname(data_dir)),'shapefiles','new_trimmed_tract',
                                   'cb_2020_tract')
town_shapefile = os.path.join(os.path.dirname(os.path.dirname(data_dir)),'shapefiles','MA_2020_Towns',
                              'CENSUS2020TOWNS_POLY.shp')
mass_gdf = gpd.read_file(new_tract_shapefile)
town_gdf = gpd.read_file(town_shapefile)


In [7]:
train_x

,geoid,lat,lon,timestep,theme_1_pc,theme_2_pc,theme_3_pc,theme_4_pc,svi_pctile,neighbor_t,self_t-1
0,25001010100,42.059829,-70.200407,0.0,0.7270,0.2972,0.5268,0.6872,0.6230,0.000,0.0
1,25001010100,42.059829,-70.200407,1.0,0.7270,0.2972,0.5268,0.6872,0.6230,0.000,0.0
2,25001010100,42.059829,-70.200407,2.0,0.7270,0.2972,0.5268,0.6872,0.6230,0.000,0.0
3,25001010100,42.059829,-70.200407,3.0,0.7270,0.2972,0.5268,0.6872,0.6230,0.000,0.0
4,25001010100,42.059829,-70.200407,4.0,0.7270,0.2972,0.5268,0.6872,0.6230,0.000,0.0
...,...,...,...,...,...,...,...,...,...,...,...
35632,25027761402,42.531342,-71.592751,14.0,0.3794,0.3189,0.4701,0.2374,0.3199,0.125,0.0
35633,25027761402,42.531342,-71.592751,15.0,0.1522,0.2139,0.3928,0.1570,0.1229,0.000,0.0
35634,25027761402,42.531342,-71.592751,16.0,0.1522,0.2139,0.3928,0.1570,0.1229,0.000,0.0
35635,25027761402,42.531342,-71.592751,17.0,0.1522,0.2139,0.3928,0.1570,0.1229,0.125,0.0


In [14]:
bprs = []
for t in [76,77,78,79]:
    y_2019 = test_y[test_y[timestep_col].isin([t])].groupby('geoid').sum()['deaths']
    last_year = train_y[train_y[timestep_col] ==18].set_index('geoid')['deaths']
    bprs.append(fixed_top_X(y_2019,last_year,100)[-1])

In [ ]:
trials_2019 = []
for _ in range(100):
    bprs = []
    for t in [76,77,78,79]:
        y_2019 = test_y[test_y[timestep_col].isin([t])].groupby('geoid').sum()['deaths']
        last_year = train_y[train_y[timestep_col] ==18].set_index('geoid')['deaths']
        bprs.append(fixed_top_X(y_2019,last_year,100)[-1])
    trials_2019.append(np.mean(bprs))

In [ ]:
bprs

In [16]:
bprs = []
for t in [76,77,78,79]:
    y_2019 = test_y[test_y[timestep_col].isin([t])].groupby('geoid').sum()['deaths']
    last_2019 = all_y_qtr[all_y_qtr[timestep_col].isin([t-1])].groupby('geoid').sum()['deaths']
    bprs.append(fixed_top_X(y_2019,last_2019,100)[-1])

In [17]:
bprs

[0.2577134502923976,
 0.24423728813559323,
 0.26881283422459895,
 0.2580765027322404]

In [ ]:
trials_2020 = []
for _ in range(100):
    bprs = []
    for t in [80,81,82,83]:
        y_2020 = test_y[test_y[timestep_col].isin([t])].groupby('geoid').sum()['deaths']
        last_year = test_y_yr[test_y_yr[timestep_col] ==19].set_index('geoid')['deaths']
        bprs.append(fixed_top_X(y_2020,last_year,100)[-1])
    trials_2020.append(np.mean(bprs))

In [23]:
np.mean(bprs)

0.3304623841419298

In [26]:
np.mean(bprs)

0.32982486060778476

In [30]:
trials_2020

[0.33027132684451355, 0.329904045222411]